In [256]:
import string
import pandas as pd
import fitz
path = "C:/DAEN690/2A13.pdf"
pdf = fitz.open(path)
#pdf.metadata

In [260]:
#### add page to all words
def add_page(in_put, value):
    lst = []
    out = []
    index = 0

    for i in range(0, len(in_put)):
        if(i < len(in_put)-1):
            out += [[]]
            lst += in_put[i]
            lst.insert( 8, value+1)
            out[index] = lst
            index += 1
            lst = []
        else:
            out +=[[]]
            lst += in_put[i]
            lst.insert( 8, value+1)
            out[index] = lst

    return out
#--------------------------------------------------------------------------------------------------------
#get all text from given pdf file
def get_all_text(data):
    page_count = data.page_count
    out = []
    for i in range(0, page_count):
        load_page = data.load_page(i)
        #WC stand for word with coordinate
        WC = load_page.get_text('words')
        agw = add_page(WC, i)
        out += agw
    
    return out
# out == [x0, y0, x1, y1, word, block_no, line_no, word_no, page]  
#---------------------------------------------------------------------------------------------------------
# merge the words by block_no and line_no
def word_line(in_put):
    in_put += [in_put[-1]]
    index = 0
    lst = []
    out = []
    for i in range(0, len(in_put)-1):
        # in_put[i][7] == word_no
        if(in_put[i][7]+1 == in_put[i+1][7]):
            lst += [in_put[i][4]]  
        else:
            if(in_put[i+1][7] == 0):
                lst += [in_put[i][4]]
                lst = [' '.join(lst)]
                lst += [in_put[i][5], in_put[i][6], in_put[i][7],in_put[i][8]]
                lst.insert( 0, in_put[i-in_put[i][7]][0])
                lst.insert( 1, in_put[i-in_put[i][7]][1])
                lst.insert( 2, in_put[i][2])
                lst.insert( 3, in_put[i][3])
            out += [[]]
            out[index] += lst
            index += 1
            lst = []
    return out
#--------------------------------------------------------------------------------------------------------
# 12/07
# Sometimes the y coordinates in the same line will be a little different,
# the smaller y, the higher position
# example: A & B in the same line 
# y0 of A ==1, y0 of B == 0.9 
# the order will be changed to B A since the y is more sensitive condition of the coming reorder work
# This function is to make the texts in the same line have the same y
def Same_y(df):
    fill = [0,9999,0,9999,0,0,0,0,0,0]
    df += [fill]
    num = 0
    big_y = 99999
    highery = big_y
    
    for i in range(0, len(df)-1):        
        # Smaller Y  == higher position in the file 
        # df[i][1] == Y0, df[i][3] == Y1
        # if the average Y of text i in the between of the Ys of text i-1, we say they are the same line 
        if((df[i][1]+df[i][3])/2 > df[i+1][1]) & ((df[i][1]+df[i][3])/2 < df[i+1][3]) & (df[i][1] < highery):
            highery = df[i][1]
            num += 1
            #print(num)
        elif((df[i][1]+df[i][3])/2 > df[i+1][1]) & ((df[i][1]+df[i][3])/2 < df[i+1][3]):
            num += 1
            #print(num)
        # if the average Y of the text i is smaller than the text i+1, we know the i+1 is next line 
        #elif((df[i][1]+df[i][3])/2 < df[i+1][1]) or (i == len(df)-1):
        else:
            if(num == 0):
                pass
            else:
                for j in range(0, num+1):
                    df[i-j][1] = highery
                num = 0
                highery = big_y
    df.pop(-1)
    

    return df    
#--------------------------------------------------------------------------------------------------------
# Get the X1 of Fuel
def get_fuel_x1(df):
    for i in range(0,len(df)):
        if(df[i][4].lower() == 'fuel') or (df[i][4].lower() == 'fuel:') or (df[i][4].lower() == '*fuel') or (df[i][4].lower() == 'fuel.'):
            return df[i][2]
#--------------------------------------------------------------------------------------------------------
# detect the word lines' class (key_term, header, info)
def detect_khi(df):
    index = 0
    lst = []
    out = []
    for i in range(0,len(df)):
        ## old one add & (in_put[i][6] == 0) in both if & elif
        if(df[i][0] <= fuel_x1) & (df[i][7] < 5 ):
            lst += df[i]
            lst.insert(8, 'key_term')
            out += [[]]
            out[index] = lst
            lst = []
            index += 1
        elif(df[i][0] <= fuel_x1) & (df[i][4].lower().find('model') > 0):
            lst += df[i]
            lst.insert(8, 'header')
            out += [[]]
            out[index] = lst
            lst = []
            index += 1
        else:
            lst += df[i]
            lst.insert(8, 'info')
            out += [[]]
            out[index] = lst
            lst = []
            index += 1

    #print(out_put)
    return out

#---------------------------------------------------------------------------------------------------------------
#12/07
#Drop no need columns (block_no, line_no, word_no)
def Drop_blw(df):
    # df == [x0, y0, x1, y1, word, block_no, line_no, word_no, category, page]    
    lst = []
    index = 0
    out = []
    for i in range(0, len(df)):
        lst += [df[i][0],df[i][1],df[i][2],df[i][3],df[i][4],df[i][8],df[i][9]]
        out += [[]]
        out[index] = lst
        index += 1
        lst = []
    # out == [x0, y0, x1, y1, word, category, page] 
    return out

#----------------------------------------------------------------------------------------------------------------
# Sometimes the header's number will be detect as key term, this function is used to solve it.
# example: real header == I. Model xyz....  and the df might be ['I.', 'key_term'], ['Model xyz...', 'header']
# hder function is to catch 'I.' form keyterm to header.       
def hder(df):
    for i in range(0, len(df)-1):
        if(df[i+1][5] == "header") & (int(df[i][1]) == int(df[i+1][1])):
            df[i][5] = "header"
    return df
#----------------------------------------------------------------------------------------------------------------

# find first line from page2
def firsty1(df):
    lst = []
    page = 1

    for i in range(0, len(df)-1):
        if(df[i][6] == page+1):
            lst += [df[i][3]]
            page += 1
    #print(lst)
    return lst

#--------------------------------------------------------------------------------------------------------
def drop_fline(df, fy):
    lst = []
    y1max = max(fy)
    for i in range(0, len(df)):
        #df[i][6] == page_no
        if(df[i][6] > 1) & (df[i][1] < y1max):
            lst += [i]
            print(df[i][4])
#    print(lst)
    lst.reverse()
#    print(lst)
    for j in range(0, len(lst)):
        df.pop(lst[j])
    
    return df
#----------------------------------------------------------------------------------------------------------------
# clean the cont'd stuff in the df
def clean_cont(df):

    lst = []
    num = 0
    page = 1 
    c1 = 'contd'
    c2 = '(cont’d)'
    c3 = '(continued)' 
    c4 = 'cont d'
    c5 = 'cont'
    for i in range(0, len(df)):
        if(df[i][6] == page) & (num < 5):
            num += 1
            if(df[i][4].lower().find(c1) > 0) or (df[i][4].lower().find(c2) > 0) or (df[i][4].lower().find(c3) > 0) or (df[i][4].lower().find(c4) > 0) or (df[i][4].lower().find(c5) > 0):
                lst += [i]
        else:
            num = 0
            page += 1
    #print(lst)
    lst.reverse()
    for j in range(0, len(lst)):
        df.pop(lst[j])
    
    return df

#----------------------------------------------------------------------------------------------------------------
# 11/15
# This function is used to make sure the left side have are all header or keyterm
def all_header(df):
    for i in range(1 , len(df)):
        if(df[i-1][5] == 'header'):
            if(df[i][4].lower() == 'engine') or (df[i][4].lower() == 'engines'):
                pass
            else:
                df[i][5] = 'header'
    return df
#----------------------------------------------------------------------------------------------------------------
#11/25
#Let 'all model' from key term to header
def allmodel_is_header(df):
    
    for i in range(0, len(df)):
        if(df[i][5] == 'key_term') & (df[i][4].lower().find('all') > 0):
            if(df[i][4].lower().find('model') > 0) or (df[i][4].lower().find('models') > 0 ):
                df[i][5] == 'header'
    
    return df
#----------------------------------------------------------------------------------------------------------------
# 11/19 define the model & revision info of top right in the first page as rev
def df_rev(df):
    fkt = -1
    for i in range(0, len(df)):
        if(df[i][5] == 'key_term') & (fkt < 0):
            fkt = i
#    print(fkt)        
    for i in range(0, fkt):
        if(df[i][0] > 300):
            df[i][5] = 'rev'
    
    return df
#----------------------------------------------------------------------------------------------------------------
def merge_all(df):
    lst = []
    out = []
    index = 0
    num = 0
    for i in range(0, len(df)-1):
        if(df[i][5] == 'header'):
            if(df[i+1][5] == 'header'):
                lst += [df[i][4]]
                num += 1
            else:
                out += [[]]
                lst += [df[i][4]]
                lst = [' \n '.join(lst)]
                lst += [df[i][5]]
                lst.insert( 0, df[i-num][6])
                lst.insert( 1, df[i][1])
                lst.insert( 2, df[i][6])
                lst.insert( 3, df[i][3])
                # out put will be [start page, y start, end page, y end, text, class]
                out[index] = lst
                num = 0
                index += 1
                lst = []

        elif(df[i][5] == 'key_term'):
            out += [[]]
            lst += [df[i][6], df[i][1], df[i][6], df[i][3], df[i][4], df[i][5]]
            out[index] = lst
            index += 1
            lst = []
        
        elif(df[i][5] == 'info'):
            if(df[i+1][5] == 'info'):
                lst += [df[i][4]]
                num += 1
            else:
                out += [[]]
                lst += [df[i][4]]
                lst = [' \n '.join(lst)]
                lst += [df[i][5]]
                lst.insert( 0, df[i-num][6])
                lst.insert( 1, df[i][1])
                lst.insert( 2, df[i][6])
                lst.insert( 3, df[i][3])
                # out put will be [start page, y start, end page, y end, text, class]
                out[index] = lst
                num = 0
                index += 1
                lst = []
                
        elif(df[i][5] == 'rev'):
            if(df[i+1][5] == 'rev'):
                lst += [df[i][4]]
                num += 1
            else:
                out += [[]]
                lst += [df[i][4]]
                lst = [' \n '.join(lst)]
                lst += [df[i][5]]
                lst.insert( 0, df[i-num][6])
                lst.insert( 1, df[i][1])
                lst.insert( 2, df[i][6])
                lst.insert( 3, df[i][3])
                # out put will be [start page, y start, end page, y end, text, class]
                out[index] = lst
                num = 0
                index += 1
                lst = []
    return out

#----------------------------------------------------------------------------------------------------------------
def prep_final_output(df):
    index = 0
    lst = []
    out = []
    
    for i in range(0, len(df)-1):
        if(df[i][5] == 'rev') or (df[i][5] == 'header'):
            out += [[]]
            lst += [df[i][4], '']
            out[index] = lst
            
            lst = []
            index += 1
        
        elif(df[i][5] == 'key_term') & (df[i+1][5] == 'info'):
            lst += [df[i][4]]
        elif(df[i][5] == 'info') & (len(lst) > 0):
            out += [[]]
            lst += [df[i][4]]
            out[index] = lst
            
            lst =[]
            index += 1

    return out

In [261]:
all_text = get_all_text(pdf)

merged_words = word_line(all_text)
#-----------------------------------
# Sometimes the y coordinates in the same line will be a little different,
# the smaller y, the higher position
# example: A & B in the same line 
# y0 of A ==1, y0 of B == 0.9 
# the order will be changed to B A since the y is more sensitive condition of the coming reorder work
# This function is to make the texts in the same line have the same y
Sy = Same_y(merged_words)

#-------------------------------------
#sorted key + lambda 
# ref: https://blog.csdn.net/w417950004/article/details/86253721
# reorder the word lines by coordinates (page > y0 > x0)
St = sorted(Sy, key = lambda x:(int(x[8]), float(x[1]), float(x[0])))

fuel_x1 = get_fuel_x1(St)
khi = detect_khi(St)
#for i in range(0, len(khi)):
#    if(khi[i][8] == 'header'):
#        print(khi[i][4])
#Cleaned khi
Ckhi = Drop_blw(khi)
#-----------------------------------------------------------------------------
# Sometimes the header's number will be detect as key term, this function is used to solve it.
#example: real header == I. Model xyz....  and the df might be ['I.', 'key_term'], ['Model xyz...', 'header']
# hder function is to catch 'I.' form keyterm to header.       
hder(Ckhi)

#-----------------------
#clean the page stuff
#fy1 function will retun the y1 of the first lines
fy1 = firsty1(Ckhi)
#use the max value to clean the tc titles which has been detect as KT
drop_fline(Ckhi, fy1)
#-----------------------------------------------------------------------------
C_cont = clean_cont(Ckhi)

#---------------------------------------------------------------------------------------------------
#drop punctuations of KT
#https://datagy.io/python-remove-punctuation-from-string/
for i in range(0, len(C_cont)-1):
    if(C_cont[i][5] == 'key_term'):
        C_cont[i][4] = C_cont[i][4].translate(str.maketrans('', '', string.punctuation))
#-------------------------------------------------------------------------------------------------
ah = all_header(C_cont)
#---------------------------------------------
allmodel_is_header(ah)
#----------------------------------------------
df_rev(ah)

mall = merge_all(ah)
#for i in range(0, len(mall)):
#    print(mall[i])
fout = prep_final_output(mall)


#for i in range(0, len(fout)):
#    print(fout[i])

dfinal = pd.DataFrame(fout, columns=['L', 'R'])
dfinal           

2A13
Page 2 of 47
Page 3 of 47
2A13
2A13
Page 4 of 47
Page 5 of 47
2A13
2A13
Page 6 of 47
Page 7 of 47
2A13
2A13
Page 8 of 47
Page 9 of 47
2A13
2A13
Page 10 of 47
Page 11 of 47
2A13
2A13
Page 12 of 47
Page 13 of 47
2A13
2A13
Page 14 of 47
Page 15 of 47
2A13
2A13
Page 16 of 47
Page 17 of 47
2A13
2A13
Page 18 of 47
Page 19 of 47
2A13
2A13
Page 20 of 47
Page 21 of 47
2A13
2A13
Page 22 of 47
Page 23 of 47
2A13
2A13
Page 24 of 47
Page 25 of 47
2A13
2A13
Page 26 of 47
Page 27 of 47
2A13
2A13
Page 28 of 47
Page 29 of 47
2A13
2A13
Page 30 of 47
Page 31 of 47
2A13
2A13
Page 32 of 47
Page 33 of 47
2A13
2A13
Page 34 of 47
Page 35 of 47
2A13
2A13
Page 36 of 47
Page 37 of 47
2A13
2A13
Page 38 of 47
Page 39 of 47
2A13
2A13
Page 40 of 47
Page 41 of 47
2A13
2A13
Page 42 of 47
Page 43 of 47
2A13
2A13
Page 44 of 47
Page 45 of 47
2A13
2A13
Page 46 of 47
Page 47 of 47
2A13


,L,R
0,"2A13 \n Revision 63 \n Piper Aircraft, Inc \n ...",
1,Type Certificate Holder,"Piper Aircraft, Inc. \n 2926 Piper Drive \n Ve..."
2,Type Certificate Holder Record,"The New Piper Aircraft, Inc transferred TC 2A1..."
3,"I - Model PA-28-160 (Cherokee), 4 PCLM (Normal...",
4,Engine,Lycoming O-320-B2B or O-320-D2A with carbureto...
...,...,...
526,NOTE 24,On Models PA-28-161; S/N 28-7816001 through 28...
527,NOTE 25,On Models PA-28-201T; S/N 28-7921001 through 2...
528,NOTE 26,"With installation of Piper Kit 88050, PA-28-16..."
529,NOTE 27,"With installation of Piper Kit 88168, PA-28-16..."


In [250]:
writer=pd.ExcelWriter('C:/DAEN690/2A13Ihopethisisthefinalfinalfinalfinalsdfsdfsdfsdfsdfsdf.xlsx',engine='xlsxwriter')
dfinal.to_excel(writer, index=False)
writer.save()